In [78]:
import torch
from NN_model import *

import sounddevice as sd
import scipy.io.wavfile as wav
from scipy.io.wavfile import write
import os.path
import numpy as np

from os.path import join, abspath, isfile, isdir, splitext
from os import makedirs, listdir

from scipy.fft import fft

split_time = 2 # Time of sample in secs

file_name = "Demonstration_F"

### Load the model

In [79]:

input_dim = 6
device = torch.device('cpu' if torch.cuda.is_available() else 'gpu')
model = Classifier(input_dim)
model.load_state_dict(torch.load('demo_model.pth'))
model = model.to(device)

RuntimeError: Error(s) in loading state_dict for Classifier:
	size mismatch for fc1.weight: copying a param with shape torch.Size([20, 11]) from checkpoint, the shape in current model is torch.Size([20, 6]).

### Record microphone data


In [73]:

# fs = 44100  # Sample rate
# seconds = split_time  # Duration of recording
# file_type = '.wav'

# file_name = file_name + file_type

# print(file_name)


# print("Begin recording")
# myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
# sd.wait()  # Wait until recording is finished
# print(len(myrecording))
# write(file_name, fs, myrecording)  # Save as WAV file



### Load microphone data

In [74]:
file = file_name + '.wav'

sr, mic_data = wav.read(file)

mic_data = mic_data[:(sr*split_time)]
mic_fft = fft(mic_data)
mic_data = 2.0/(mic_data.shape[0]) * np.abs(mic_fft[0:mic_data.shape[0]//2]) 

### Get dSpace data

In [75]:


file = file_name + '.csv'

cur_samplerate = 1000

arr = np.loadtxt(file,
            delimiter=",", dtype=str, usecols=(1,2,3,4,5,6), skiprows=16)
data = arr[12:] # data consists of ['time','I_bat[A]', 'I_dyno_LP[A]', 'i_motor_LP[A]','speed_motor[rads]', 'u_bat[V]']
data = data.astype(float)

cur_data = data[:,3]# Only obtain current running through motor

cur_data = cur_data[:(split_time * cur_samplerate)]


### Get statistical features from measurement

In [76]:

speed = 20

mean_mic = np.mean(mic_data)
stds_mic = np.std(mic_data)
maxs_mic = np.max(mic_data)
mins_mic = np.min(mic_data)
medians_mic = np.median(mic_data)

mean_cur = np.float32(np.mean(cur_data))
stds_cur = np.float32(np.std(cur_data))
maxs_cur = np.float32(np.max(cur_data))
mins_cur = np.float32(np.min(cur_data))
medians_cur = np.float32(np.median(cur_data))


inputs = torch.tensor([speed, mean_mic, stds_mic, maxs_mic, mins_mic, medians_mic])#, mean_cur, stds_cur, maxs_cur, mins_cur, medians_cur])



### Make prediction

In [77]:

with torch.no_grad():
  pred = model(inputs)

predicted = (pred >= 0.5).squeeze().long()

print("Drive is anomalous") if predicted == 1 else print("Drive is normal") 


Drive is anomalous
